In [105]:
import pandas as pd 
import numpy as np 
import geopandas as gpd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import censusdata
warnings.simplefilter(action="ignore", category=FutureWarning)
%matplotlib inline

import requests

url = 'https://api.census.gov/data/2019/acs/acs5?get=B01001_001E,B03002_012E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E,B02001_007E,GEO_ID,NAME&for=tract:*&in=state:36'

response = requests.get(url)
data = response.json()

df = pd.DataFrame(data[1:], columns=data[0])

In [110]:
hispanic_race_columns = []

for i in range(1,13):
    if i == 2 or i == 10 or i == 11:
        pass
    elif i <10:
        hispanic_race_columns.append('B03002_00{}E'.format(i))
    else:
        hispanic_race_columns.append('B03002_0{}E'.format(i))


column_vals = ['Total','White', 'Black or African American','American Indian and Alaska Native', 'Asain',\
               'Native Hawaiian and Other Pacific Islander', 'Some other race alone', 'Two or More Races']

hisp_col_vals = ['Total'] + [item+', not hispanic' for item in column_vals[1:]] +['Hispanic or Latino']
col_dict = {hispanic_race_columns[i]: hisp_col_vals[i] for i in range(len(hisp_col_vals))}
print(col_dict)

counties = censusdata.geographies(censusdata.censusgeo([('state', '36'), ('county','*'), ('tract', '*')]), 'acs5', 2020)

{'B03002_001E': 'Total', 'B03002_003E': 'White, not hispanic', 'B03002_004E': 'Black or African American, not hispanic', 'B03002_005E': 'American Indian and Alaska Native, not hispanic', 'B03002_006E': 'Asain, not hispanic', 'B03002_007E': 'Native Hawaiian and Other Pacific Islander, not hispanic', 'B03002_008E': 'Some other race alone, not hispanic', 'B03002_009E': 'Two or More Races, not hispanic', 'B03002_012E': 'Hispanic or Latino'}


In [116]:
def get_acs5_data(yr,stateid,countyids,column_ls,col_dict):
    """
    Parameters
    ----------
    yr: int
        The census year of interest
    stateid: str
        State id  of interest in the form of a string
    countyids: list of str
        County ids of interest in the form of a list of strings
    column_ls: list
        The list of ACS column values you are looking to extract
    col_dict: dict
        Dictionary assigning categorical values to associated ACS column
        
        key: ACS column (e.g. B3002_001E)
        value: Categorical value (e.g. 'Total')
        
    Returns
    -------
    df: dataframe
        Dataframe containing population counts of interest at the available
        geographical/spatial resolution (e.g. census blocks, tracts, block groups)
    """
    dfs = []
    for countyid in countyids:
        df = censusdata.download('acs5', yr,
                   censusdata.censusgeo([('state', stateid),
                                         ('county', countyid),
                                         ('block group', '*')]),
                                          column_ls)
        df.reset_index(inplace=True)
        df['index'] = df['index'].astype('str')

        Block_Group = []
        tract = []
        for index, row in df.iterrows():
            Block_Group.append(row['index'][:13])
            tract.append(row['index'][-21:-15])

        df['Block Group'] = Block_Group
        df['Tract'] = tract

        df = df.rename(columns = col_dict)
        df = df.rename(columns = {'index':'Geographical Information'})
        dfs.append(df)
        
    return pd.concat(dfs)


In [117]:
countyids = ['047', '061', '081', '085', '005']
eth = get_acs5_data(2020, '36', countyids, hispanic_race_columns, col_dict)
eth.head()

,Geographical Information,Total,"White, not hispanic","Black or African American, not hispanic","American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino,Block Group,Tract
0,"Block Group 1, Census Tract 133, Kings County,...",895,796,31,0,0,0,0,17,51,Block Group 1,013300
1,"Block Group 4, Census Tract 133, Kings County,...",917,733,33,0,89,0,0,62,0,Block Group 4,013300
2,"Block Group 3, Census Tract 134, Kings County,...",1566,1129,9,0,166,0,0,8,254,Block Group 3,013400
3,"Block Group 3, Census Tract 135, Kings County,...",1389,830,0,0,205,0,12,222,120,Block Group 3,013500
4,"Block Group 3, Census Tract 136, Kings County,...",1448,1170,0,0,72,0,0,33,173,Block Group 3,013600


In [120]:
eth['Geographical Information']

0       Block Group 1, Census Tract 133, Kings County,...
1       Block Group 4, Census Tract 133, Kings County,...
2       Block Group 3, Census Tract 134, Kings County,...
3       Block Group 3, Census Tract 135, Kings County,...
4       Block Group 3, Census Tract 136, Kings County,...
                              ...                        
1177    Block Group 2, Census Tract 221.02, Bronx Coun...
1178    Block Group 3, Census Tract 221.02, Bronx Coun...
1179    Block Group 1, Census Tract 222, Bronx County,...
1180    Block Group 3, Census Tract 222, Bronx County,...
1181    Block Group 4, Census Tract 222, Bronx County,...
Name: Geographical Information, Length: 6807, dtype: object

In [ ]:
Block Group 1, Census Tract 133, Kings County

In [123]:
eth[['Blockgroup', 'Census Tract', 'County Name', 'State', 'state']] = eth['Geographical Information'].str.split(',', expand=True)

In [124]:
eth

,Geographical Information,Total,"White, not hispanic","Black or African American, not hispanic","American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino,Block Group,Tract,Blockgroup,Census Tract,County Name,State,state
0,"Block Group 1, Census Tract 133, Kings County,...",895,796,31,0,0,0,0,17,51,Block Group 1,013300,Block Group 1,Census Tract 133,Kings County,New York: Summary level: 150,state:36> county:047> tract:013300> block gro...
1,"Block Group 4, Census Tract 133, Kings County,...",917,733,33,0,89,0,0,62,0,Block Group 4,013300,Block Group 4,Census Tract 133,Kings County,New York: Summary level: 150,state:36> county:047> tract:013300> block gro...
2,"Block Group 3, Census Tract 134, Kings County,...",1566,1129,9,0,166,0,0,8,254,Block Group 3,013400,Block Group 3,Census Tract 134,Kings County,New York: Summary level: 150,state:36> county:047> tract:013400> block gro...
3,"Block Group 3, Census Tract 135, Kings County,...",1389,830,0,0,205,0,12,222,120,Block Group 3,013500,Block Group 3,Census Tract 135,Kings County,New York: Summary level: 150,state:36> county:047> tract:013500> block gro...
4,"Block Group 3, Census Tract 136, Kings County,...",1448,1170,0,0,72,0,0,33,173,Block Group 3,013600,Block Group 3,Census Tract 136,Kings County,New York: Summary level: 150,state:36> county:047> tract:013600> block gro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1177,"Block Group 2, Census Tract 221.02, Bronx Coun...",1408,10,0,0,19,0,0,0,1379,Block Group 2,022102,Block Group 2,Census Tract 221.02,Bronx County,New York: Summary level: 150,state:36> county:005> tract:022102> block gro...
1178,"Block Group 3, Census Tract 221.02, Bronx Coun...",1919,101,344,0,0,0,49,51,1374,Block Group 3,022102,Block Group 3,Census Tract 221.02,Bronx County,New York: Summary level: 150,state:36> county:005> tract:022102> block gro...
1179,"Block Group 1, Census Tract 222, Bronx County,...",968,54,45,0,411,0,13,43,402,Block Group 1,022200,Block Group 1,Census Tract 222,Bronx County,New York: Summary level: 150,state:36> county:005> tract:022200> block gro...
1180,"Block Group 3, Census Tract 222, Bronx County,...",602,6,125,77,135,0,0,21,238,Block Group 3,022200,Block Group 3,Census Tract 222,Bronx County,New York: Summary level: 150,state:36> county:005> tract:022200> block gro...


In [125]:
eth.columns

Index(['Geographical Information', 'Total', 'White, not hispanic',
       'Black or African American, not hispanic',
       'American Indian and Alaska Native, not hispanic',
       'Asain, not hispanic',
       'Native Hawaiian and Other Pacific Islander, not hispanic',
       'Some other race alone, not hispanic',
       'Two or More Races, not hispanic', 'Hispanic or Latino', 'Block Group',
       'Tract', 'Blockgroup', 'Census Tract', 'County Name', 'State', 'state'],
      dtype='object')

In [126]:
eth = eth.drop(['Geographical Information','Block Group', 'Blockgroup', 'Census Tract', 'County Name', 'State', 'state'], axis=1)

In [127]:
eth

,Total,"White, not hispanic","Black or African American, not hispanic","American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino,Tract
0,895,796,31,0,0,0,0,17,51,013300
1,917,733,33,0,89,0,0,62,0,013300
2,1566,1129,9,0,166,0,0,8,254,013400
3,1389,830,0,0,205,0,12,222,120,013500
4,1448,1170,0,0,72,0,0,33,173,013600
...,...,...,...,...,...,...,...,...,...,...
1177,1408,10,0,0,19,0,0,0,1379,022102
1178,1919,101,344,0,0,0,49,51,1374,022102
1179,968,54,45,0,411,0,13,43,402,022200
1180,602,6,125,77,135,0,0,21,238,022200


In [128]:
geo = gpd.read_file('/Users/ananyarajesh/Downloads/nyct2020_23a/nyct2020.shp')

In [129]:
gdf = gpd.read_file('/Users/ananyarajesh/Downloads/nynta2020_23a/nynta2020.shp')

In [130]:
geo.head()

,CTLabel,BoroCode,BoroName,CT2020,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,GEOID,Shape_Leng,Shape_Area,geometry
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,11023.048501,1.844421e+06,"MULTIPOLYGON (((972081.788 190733.467, 972184...."
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4754.495247,9.723121e+05,"POLYGON ((988548.218 197770.375, 987978.808 19..."
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,6976.286456,2.582705e+06,"POLYGON ((986961.185 199553.643, 987206.139 19..."
3,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,"POLYGON ((987475.016 200297.218, 987705.443 20..."
4,14.02,1,Manhattan,001402,1001402,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001402,4459.156019,1.226206e+06,"POLYGON ((988387.669 201258.312, 988621.002 20..."


In [131]:
geo.rename(columns={'CT2020': 'Tract'}, inplace=True)

In [132]:
ct = pd.merge(eth, geo, on='Tract')

In [133]:
ct

,Total,"White, not hispanic","Black or African American, not hispanic","American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino,Tract,...,BoroCT2020,CDEligibil,NTAName,NTA2020,CDTA2020,CDTANAME,GEOID,Shape_Leng,Shape_Area,geometry
0,895,796,31,0,0,0,0,17,51,013300,...,1013300,None,Hell's Kitchen,MN0402,MN04,MN04 Chelsea-Hell's Kitchen (CD 4 Approximation),36061013300,5684.395759,1.876670e+06,"POLYGON ((988603.981 217911.418, 988478.912 21..."
1,895,796,31,0,0,0,0,17,51,013300,...,2013300,None,Morrisania,BX0301,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,36005013300,5723.849903,1.734820e+06,"POLYGON ((1011600.676 239639.743, 1011424.125 ..."
2,895,796,31,0,0,0,0,17,51,013300,...,3013300,None,Park Slope,BK0602,BK06,BK06 Park Slope-Carroll Gardens (CD 6 Approxim...,36047013300,5175.703511,1.596011e+06,"POLYGON ((990363.521 185357.462, 990244.214 18..."
3,917,733,33,0,89,0,0,62,0,013300,...,1013300,None,Hell's Kitchen,MN0402,MN04,MN04 Chelsea-Hell's Kitchen (CD 4 Approximation),36061013300,5684.395759,1.876670e+06,"POLYGON ((988603.981 217911.418, 988478.912 21..."
4,917,733,33,0,89,0,0,62,0,013300,...,2013300,None,Morrisania,BX0301,BX03,BX03 Morrisania-Crotona Park East (CD 3 Approx...,36005013300,5723.849903,1.734820e+06,"POLYGON ((1011600.676 239639.743, 1011424.125 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14161,235,0,0,0,0,0,0,0,235,021301,...,2021301,None,University Heights (South)-Morris Heights,BX0501,BX05,BX05 Morris Heights-Mount Hope (CD 5 Approxima...,36005021301,3825.333710,6.609102e+05,"POLYGON ((1007066.374 247627.824, 1007091.876 ..."
14162,940,66,307,0,0,16,0,0,551,021301,...,2021301,None,University Heights (South)-Morris Heights,BX0501,BX05,BX05 Morris Heights-Mount Hope (CD 5 Approxima...,36005021301,3825.333710,6.609102e+05,"POLYGON ((1007066.374 247627.824, 1007091.876 ..."
14163,592,8,66,0,18,0,135,0,365,022101,...,2022101,None,Mount Eden-Claremont (West),BX0403,BX04,BX04 Highbridge-Concourse (CD 4 Approximation),36005022101,3869.863198,7.322451e+05,"POLYGON ((1007509.408 245159.939, 1007374.372 ..."
14164,1080,8,103,0,0,0,7,32,930,022101,...,2022101,None,Mount Eden-Claremont (West),BX0403,BX04,BX04 Highbridge-Concourse (CD 4 Approximation),36005022101,3869.863198,7.322451e+05,"POLYGON ((1007509.408 245159.939, 1007374.372 ..."


In [134]:
ct.columns

Index(['Total', 'White, not hispanic',
       'Black or African American, not hispanic',
       'American Indian and Alaska Native, not hispanic',
       'Asain, not hispanic',
       'Native Hawaiian and Other Pacific Islander, not hispanic',
       'Some other race alone, not hispanic',
       'Two or More Races, not hispanic', 'Hispanic or Latino', 'Tract',
       'CTLabel', 'BoroCode', 'BoroName', 'BoroCT2020', 'CDEligibil',
       'NTAName', 'NTA2020', 'CDTA2020', 'CDTANAME', 'GEOID', 'Shape_Leng',
       'Shape_Area', 'geometry'],
      dtype='object')

In [135]:
ct = ct.drop(['CTLabel', 'BoroCode', 'BoroName', 'BoroCT2020', 'CDEligibil', 'CDTA2020', 'CDTANAME','Shape_Leng', 'Shape_Area'], axis=1)

In [136]:
ct.head()

,Total,"White, not hispanic","Black or African American, not hispanic","American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino,Tract,NTAName,NTA2020,GEOID,geometry
0,895,796,31,0,0,0,0,17,51,013300,Hell's Kitchen,MN0402,36061013300,"POLYGON ((988603.981 217911.418, 988478.912 21..."
1,895,796,31,0,0,0,0,17,51,013300,Morrisania,BX0301,36005013300,"POLYGON ((1011600.676 239639.743, 1011424.125 ..."
2,895,796,31,0,0,0,0,17,51,013300,Park Slope,BK0602,36047013300,"POLYGON ((990363.521 185357.462, 990244.214 18..."
3,917,733,33,0,89,0,0,62,0,013300,Hell's Kitchen,MN0402,36061013300,"POLYGON ((988603.981 217911.418, 988478.912 21..."
4,917,733,33,0,89,0,0,62,0,013300,Morrisania,BX0301,36005013300,"POLYGON ((1011600.676 239639.743, 1011424.125 ..."


In [137]:
gdf.head()

,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Shape_Leng,Shape_Area,geometry
0,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28914.172236,3.532185e+07,"POLYGON ((1003059.997 204572.025, 1002991.367 ..."
1,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28158.316197,2.886214e+07,"POLYGON ((994849.011 203499.267, 994911.093 20..."
2,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),18250.280262,1.520896e+07,"POLYGON ((998047.210 196303.325, 998157.901 19..."
3,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),43184.772815,5.226621e+07,"POLYGON ((1005302.497 199455.730, 1005307.792 ..."
4,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,14312.504911,9.982322e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."


In [138]:
nta = pd.merge(gdf, ct, on='NTA2020')

In [139]:
nta

,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName_x,NTAAbbrev,NTAType,CDTA2020,CDTAName,Shape_Leng,...,"American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino,Tract,NTAName_y,GEOID,geometry_y
0,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28914.172236,...,0,55,0,0,79,33,049900,Greenpoint,36047049900,"POLYGON ((999660.645 201684.423, 999328.654 20..."
1,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28914.172236,...,0,38,0,6,231,125,049900,Greenpoint,36047049900,"POLYGON ((999660.645 201684.423, 999328.654 20..."
2,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28914.172236,...,0,0,0,0,0,0,049900,Greenpoint,36047049900,"POLYGON ((999660.645 201684.423, 999328.654 20..."
3,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28914.172236,...,0,1144,0,0,20,306,049900,Greenpoint,36047049900,"POLYGON ((999660.645 201684.423, 999328.654 20..."
4,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),28914.172236,...,193,1246,0,43,258,584,049900,Greenpoint,36047049900,"POLYGON ((999660.645 201684.423, 999328.654 20..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14161,5,Staten Island,085,SI9593,Great Kills Park,GrtKlsPk,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,41465.430028,...,0,0,0,0,0,0,015400,Great Kills Park,36085015400,"POLYGON ((952739.319 138974.819, 952732.507 13..."
14162,5,Staten Island,085,SI9593,Great Kills Park,GrtKlsPk,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,41465.430028,...,0,200,0,103,140,117,015400,Great Kills Park,36085015400,"POLYGON ((952739.319 138974.819, 952732.507 13..."
14163,5,Staten Island,085,SI9593,Great Kills Park,GrtKlsPk,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,41465.430028,...,0,573,0,340,26,80,015400,Great Kills Park,36085015400,"POLYGON ((952739.319 138974.819, 952732.507 13..."
14164,5,Staten Island,085,SI9593,Great Kills Park,GrtKlsPk,9,SI95,SI95 Great Kills Park-Fort Wadsworth (JIA 95 A...,41465.430028,...,0,0,0,0,0,0,015400,Great Kills Park,36085015400,"POLYGON ((952739.319 138974.819, 952732.507 13..."


In [141]:
nta = nta.drop(['BoroCode', 'BoroName', 'CountyFIPS', 'NTAName_x',
       'NTAAbbrev', 'NTAType', 'CDTA2020', 'CDTAName', 'Shape_Leng',
       'Shape_Area', 'geometry_x'], axis=1)

In [151]:
merge = nta.groupby('NTAName_y')['Total', 'White, not hispanic',
       'Black or African American, not hispanic',
       'American Indian and Alaska Native, not hispanic',
       'Asain, not hispanic',
       'Native Hawaiian and Other Pacific Islander, not hispanic',
       'Some other race alone, not hispanic',
       'Two or More Races, not hispanic', 'Hispanic or Latino'].sum().reset_index()

In [152]:
merge

,NTAName_y,Total,"White, not hispanic","Black or African American, not hispanic","American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino
0,Allerton,62155,9176,23724,68,2927,5,580,1285,24390
1,Alley Pond Park,90,59,3,0,6,0,0,0,22
2,Annadale-Huguenot-Prince's Bay-Woodrow,53872,39204,4009,136,4546,0,550,820,4607
3,Arden Heights-Rossville,30539,23053,163,20,2407,27,49,271,4549
4,Astoria (Central),129518,62414,16345,227,12030,9,536,3462,34495
...,...,...,...,...,...,...,...,...,...,...
257,Windsor Terrace-South Slope,51440,34013,2687,0,5020,0,295,1718,7707
258,Woodhaven,114909,19534,8434,532,35803,0,995,1851,47760
259,Woodlawn Cemetery,0,0,0,0,0,0,0,0,0
260,Woodside,197137,32794,33585,275,25843,18,980,3147,100495


In [153]:
merge.columns

Index(['NTAName_y', 'Total', 'White, not hispanic',
       'Black or African American, not hispanic',
       'American Indian and Alaska Native, not hispanic',
       'Asain, not hispanic',
       'Native Hawaiian and Other Pacific Islander, not hispanic',
       'Some other race alone, not hispanic',
       'Two or More Races, not hispanic', 'Hispanic or Latino'],
      dtype='object')

In [154]:
race_cols = ['White, not hispanic',
       'Black or African American, not hispanic',
       'American Indian and Alaska Native, not hispanic',
       'Asain, not hispanic',
       'Native Hawaiian and Other Pacific Islander, not hispanic',
       'Some other race alone, not hispanic',
       'Two or More Races, not hispanic', 'Hispanic or Latino']
# Convert raw counts to percentages
for col in race_cols:
    merge[col] = merge[col] / merge['Total'] * 100

In [155]:
merge

,NTAName_y,Total,"White, not hispanic","Black or African American, not hispanic","American Indian and Alaska Native, not hispanic","Asain, not hispanic","Native Hawaiian and Other Pacific Islander, not hispanic","Some other race alone, not hispanic","Two or More Races, not hispanic",Hispanic or Latino
0,Allerton,62155,14.763092,38.169093,0.109404,4.709195,0.008044,0.933151,2.067412,39.240608
1,Alley Pond Park,90,65.555556,3.333333,0.000000,6.666667,0.000000,0.000000,0.000000,24.444444
2,Annadale-Huguenot-Prince's Bay-Woodrow,53872,72.772498,7.441714,0.252450,8.438521,0.000000,1.020939,1.522127,8.551752
3,Arden Heights-Rossville,30539,75.487082,0.533744,0.065490,7.881725,0.088412,0.160451,0.887390,14.895707
4,Astoria (Central),129518,48.189441,12.619868,0.175265,9.288284,0.006949,0.413842,2.672988,26.633364
...,...,...,...,...,...,...,...,...,...,...
257,Windsor Terrace-South Slope,51440,66.121695,5.223561,0.000000,9.758942,0.000000,0.573484,3.339813,14.982504
258,Woodhaven,114909,16.999539,7.339721,0.462975,31.157699,0.000000,0.865903,1.610840,41.563324
259,Woodlawn Cemetery,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,Woodside,197137,16.635132,17.036376,0.139497,13.109158,0.009131,0.497116,1.596352,50.977239


In [156]:
#merge.to_csv('Race.csv', index=False)